In [102]:
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader
import pandas as pd

In [103]:
# Read Data
spectrum_train = pd.read_excel('spectrum_train.xlsx')
spectrum_test = pd.read_excel('spectrum_valid.xlsx')
temp_train = pd.read_excel('temp_train.xlsx')
temp_test = pd.read_excel('temp_valid.xlsx')

spectrum_train_scaled = spectrum_train.multiply(10**12)
spectrum_test_scaled = spectrum_test.multiply(10**12)

In [104]:
temp_to_spec = False

if temp_to_spec:
    X_train = temp_train
    X_test = temp_test
    y_train = spectrum_train_scaled
    y_test = spectrum_test_scaled
else:
    X_train = spectrum_train_scaled
    X_test = spectrum_test_scaled
    y_train = temp_train
    y_test = temp_test

In [105]:
# determine the supported device
def get_device():
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu') # don't have GPU 
    return device

# convert a df to tensor to be used in pytorch
def df_to_tensor(df):
    device = get_device()
    return torch.from_numpy(df.values).float().to(device)

In [106]:
input_data = df_to_tensor(X_train)
output_data = df_to_tensor(y_train)

In [107]:
class Net(nn.Module):
    def __init__(self, input_size, output_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 256)
        self.fc3 = nn.Linear(256, 512)
        self.fc4 = nn.Linear(512, 256)
        self.fc5 = nn.Linear(256, 128)
        self.fc6 = nn.Linear(128, output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = torch.relu(self.fc5(x))
        x = self.fc6(x)
        return x

In [108]:
input_size = len(X_train.columns)
output_size = len(y_train.columns)
model = Net(input_size=input_size, output_size=output_size)

In [109]:
criterion = nn.MSELoss()  # Mean Squared Error loss for regression
optimizer = optim.Rprop(model.parameters(), lr=0.001) 

In [110]:
num_epochs = 15
batch_size = 5
num_batches = len(input_data) // batch_size

for epoch in range(num_epochs):
    for batch in range(num_batches):
        start = batch * batch_size
        end = start + batch_size

        inputs = input_data[start:end]
        targets = output_data[start:end]

        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Compute loss
        loss = criterion(outputs, targets)

        # Backpropagation and optimization
        loss.backward()
        optimizer.step()

    # Print the loss for this epoch
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.6f}")

Epoch 1/15, Loss: 4928.372559
Epoch 2/15, Loss: 6398.614746
Epoch 3/15, Loss: 2666.708008
Epoch 4/15, Loss: 4109.602539
Epoch 5/15, Loss: 3477.305664
Epoch 6/15, Loss: 4977.424805
Epoch 7/15, Loss: 9113.625977
Epoch 8/15, Loss: 1128.499023
Epoch 9/15, Loss: 7149.937500
Epoch 10/15, Loss: 8983.189453
Epoch 11/15, Loss: 6523.916992
Epoch 12/15, Loss: 7071.606934
Epoch 13/15, Loss: 4571.561035
Epoch 14/15, Loss: 3414.614258
Epoch 15/15, Loss: 5148.553223


In [111]:
predictions = model(df_to_tensor(X_test))
actual = df_to_tensor(y_test)

In [116]:
print(predictions.shape)
print(actual.shape)

torch.Size([192, 11])
torch.Size([192, 11])


In [112]:
from torchmetrics import R2Score

r2score = R2Score()
r2score(predictions, actual)

RuntimeError: output with shape [1] doesn't match the broadcast shape [11]